In [1]:
from dotenv import load_dotenv
_ = load_dotenv("config.env")

from common.mongodb import *
from common.files import *


drop_collection=True
collection = get_mongo_collection("filter_perfomance_test", drop_collection=drop_collection)

# new mock data
if drop_collection:
    insert_into_mongo(collection, 10)

def exec_query(query, collection):
    t1 = datetime.datetime.now()
    query_result = collection.find(query)
    list_query_result = list(query_result)
    len_list = len(list_query_result)
    t2 = datetime.datetime.now()
    dt = round(((t2-t1).total_seconds()*1000), 2)
    print(f"Query took {dt} ms and has {len_list} results.")

collection.find_one()

host: localhost
Inserted 10 documents into MongoDB.


{'_id': '29cd9ac9-211d-472f-bc2b-8821cc5fda1f',
 'filepath': 'fp0_2/fp1_4/fp2_3/File_9214.docx',
 'filepath_custom': 'fp_c0_2/fp_c1_2/fp_c2_6/File_9214.docx',
 'category': 'Rechnung',
 'src': {'type': 'local', 'fp': None},
 'metadata': {'infos': {'receiptid': 'rechnung 6897c7f0-6df1-410f-b6f2-8c304fcf3afe',
   'receiptdate': datetime.datetime(2024, 7, 7, 18, 35, 12, 530000)},
  'category': {'maincategory': 'capital', 'subcategory': 'example'},
  'buyer': {'firstname': 'Mathew',
   'lastname': 'Gallegos',
   'company': 'Collins-Hamilton'},
  'kseller': {'firstname': 'Steven',
   'lastname': 'Brandt',
   'company': 'Smith-Chen'},
  'price': {'price': 156.33}},
 'summary': 'Small summary of the file ... .'}

In [2]:
# normal filter
# many results
query = { "metadata.infos.receiptid": {"$regex": "^rechnung"}}
exec_query(query, collection)

# only few results
query = { "metadata.infos.receiptid": {"$regex": "^rechnung 474e10b5-a8eb-4396-9"}}
exec_query(query, collection)

# no results
query = { "metadata.infos.dummyreceiptid": {"$regex": "^rechnung"}}
exec_query(query, collection)

# show no caching
query = { "metadata.infos.receiptid": {"$regex": "^rechnung"}}
exec_query(query, collection)

Query took 1.5 ms and has 10 results.
Query took 0.6 ms and has 0 results.
Query took 1.76 ms and has 0 results.
Query took 0.53 ms and has 10 results.


In [3]:
# numbers
query = {"metadata.price.price": {"$gt": 0}}
exec_query(query, collection)

query = {"metadata.price.price": {"$gt": 998}}
exec_query(query, collection)

Query took 0.93 ms and has 10 results.
Query took 0.55 ms and has 0 results.


In [4]:
# dates

query = {
    "$expr": {"$lt": ["$metadata.infos.receiptdate", datetime.datetime(9999, 1, 1)]}
}
exec_query(query, collection)

query = {"$expr": {"$eq": [{"$month": "$metadata.infos.receiptdate"}, 10]}}
exec_query(query, collection)

Query took 1.13 ms and has 10 results.
Query took 0.47 ms and has 0 results.


In [5]:
# and/or
query = {
    "$and": [
        {"metadata.infos.receiptid": {"$regex": "^rechnung"}},
        {"metadata.seller.company": {"$regex": "^L"}},
        {
            "$or": [
                {"metadata.price.price": {"$lt": 8200}},
            ]
        },
    ]
}

exec_query(query, collection)

Query took 0.55 ms and has 0 results.


In [6]:
# compound index
collection.create_index([
    ("metadata.infos.receiptid", "text"),
    ("metadata.category.maincategory", "text")
])
query = {"$text": {"$search": "rechnung"}}
exec_query(query, collection)

Query took 4.83 ms and has 10 results.
